# Load polygon values

This is a demo that queries a polygon and evaluates near- and long-term risks to extreme heat based on the future [Extreme Heat](https://data.spatiafi.com/dataset/13) datasets.  We start by loading our App Credentials, and then using the [geojson statistics](https://docs.spatiafi.com/api/#operation/geojson_statistics_api_statistics_post) endpoint to fetch extreme heat risk values at the points of interest.

---

Install `spatiafi` (if not already installed):

In [ ]:
!sudo apt -y install libgeos-dev

In [ ]:
%pip install --upgrade spatiafi cartopy xarray

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import spatiafi

In [ ]:
session = spatiafi.get_session()

In [ ]:
session

## Polygon

We first choose an area around Northern Saudi Arabia and query the year 2050 SSP5-8.5 heat risk probability band for all locations within this region.

In [ ]:
url = "https://api.spatiafi.com/api/statistics"

params = {
    "item_id": "ce-extreme-heat-risk-projection-global-v1.0-ssp585-2020",
    "bidx": "1",
}

# North Saudi Arabia
payload = {
    "type": "Feature",
    "properties": {},
    "geometry": {
        "coordinates": [
            [
                [31.76491069149293, 32.29986592075784],
                [31.76491069149293, 23.092931581000542],
                [51.58140483659537, 23.092931581000542],
                [51.58140483659537, 32.29986592075784],
                [31.76491069149293, 32.29986592075784],
            ]
        ],
        "type": "Polygon",
    },
}

# `POST` the request using our `session` object, which will automatically handle authentication.
response = session.post(url, json=payload, params=params)

data = response.json()
print(data)

From this, we can see that we have 16,337 grid points with heat risk values defined in this region. Their values range from 0 to ~44% chance that there is extreme heat in any given location.

## Polygon Timeseries

We then query the extreme heat data over the same region but defined as a timeseries from 2020 to 2100.

In [ ]:
url = "https://api.spatiafi.com/api/timeseries/statistics"

params = {
    "coll_id": "spatiafi-extreme-heat-risk-projections-global-v1.0-ssp585",
    "datetime": "2020/2100",
    "bidx": "1",
}

# North Saudi Arabia
payload = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "geometry": {
                "coordinates": [
                    [
                        [31.76491069149293, 32.29986592075784],
                        [31.76491069149293, 23.092931581000542],
                        [51.58140483659537, 23.092931581000542],
                        [51.58140483659537, 32.29986592075784],
                        [31.76491069149293, 32.29986592075784],
                    ]
                ],
                "type": "Polygon",
            },
            "properties": {},
            "id": 0,
        }
    ],
}

# `POST` the request using our `session` object, which will automatically handle authentication.
response = session.post(url, json=payload, params=params)
response.raise_for_status()

data = response.json()

In [ ]:
# Extract the dates and mean values
dates = []
mean_values = []

for feature in data["features"]:
    for date, stats in feature["properties"]["statistics"].items():
        dates.append(date)
        mean_values.append(stats["mean"])

# Create a pandas DataFrame
df = pd.DataFrame({"Date": dates, "Mean Value": mean_values})
df

So the extreme heat index in this region of Saudi Arabia is expected to more than triple by 2090 under a pessimistic emissions scenario. We then turn this into a pandas date and mean values for each decade into a pandas dataframe and plot the data.

In [ ]:
# Extract the dates and mean values
dates = []
mean_values = []

for feature in data["features"]:
    for date, stats in feature["properties"]["statistics"].items():
        dates.append(date)
        mean_values.append(stats["mean"])

# Create a pandas DataFrame
df = pd.DataFrame({"Date": dates, "Mean Value": mean_values})

# Sort the DataFrame by 'Date' in ascending order
df = df.sort_values(by="Date")
df["Date"] = pd.to_datetime(df["Date"])
df

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(df["Date"], df["Mean Value"], marker="o", linestyle="-", color="b")
plt.title("Time Series Plot of Mean Prob. of Extreme Heat over time in SSP5-8.5")
plt.xlabel("Year")
plt.ylabel("Mean Value")
plt.grid(True)
plt.xticks(df["Date"], df["Date"].dt.year, rotation=45)  # Display only year in x-labels
plt.tight_layout()

plt.show()

So the mean probability of extreme heat in Northern Saudi Arabia is expected to triple by the end of century in SSP5-8.5.